In [6]:
import torch
import rasterio
import os

inferences = []

# Define the directory path where the rasters are located
directory = '/nfs/home/innocenti/igarss-fmars/output_images/mic_sampler_09'
directories = os.listdir(directory)
# sort
directories.sort()
# for each directory in the directory
for folder in directories:
    folder = os.path.join(directory, folder)
    # append 'pre/'
    folder = os.path.join(folder, 'pre/')
    for mosaic in os.listdir(folder):
        mosaic = os.path.join(folder, mosaic)
        for tile in os.listdir(mosaic):
            tile = os.path.join(mosaic, tile)
            # ceck if tile is a tif
            if tile.endswith('.tif'):
                # Open the raster file
                with rasterio.open(tile) as src:
                    # Read the raster data
                    raster_data = src.read(1)
                    # Convert the raster data to a torch tensor
                    inferences.append(torch.from_numpy(raster_data))

/nfs/home/innocenti/igarss-fmars/.venv2/lib/python3.8/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [7]:
print(len(inferences))
print(inferences[0].shape)
print(inferences[0].unique())

19
torch.Size([17408, 17408])
tensor([0, 1, 2, 3], dtype=torch.uint8)


In [8]:
labels = []

# Define the directory path where the rasters are located
directory = '/nfs/home/innocenti/igarss-fmars/data/outputs/04_05/test'
directories = os.listdir(directory)
# sort
directories.sort()
# for each directory in the directory
for folder in directories:
    folder = os.path.join(directory, folder)
    # append 'pre/'
    folder = os.path.join(folder, 'pre/')
    for mosaic in os.listdir(folder):
        mosaic = os.path.join(folder, mosaic)
        for tile in os.listdir(mosaic):
            tile = os.path.join(mosaic, tile)
            # ceck if tile is a tif
            if tile.endswith('.tif'):
                # Open the raster file
                with rasterio.open(tile) as src:
                    # Read the raster data
                    raster_data = src.read(1)
                    # shift all labels by 1, set 255 to 1
                    raster_data = raster_data + 1
                    raster_data[raster_data == 256] = 0
                    # Convert the raster data to a torch tensor
                    labels.append(torch.from_numpy(raster_data))

In [9]:
print(len(labels))
print(labels[0].shape)
print(labels[0].unique())

19
torch.Size([17408, 17408])
tensor([0, 1, 2, 3], dtype=torch.uint8)


In [10]:
from torchmetrics import Accuracy, JaccardIndex

# Convert the labels and inferences to torch tensors
labels_tensor = torch.stack(labels)
inferences_tensor = torch.stack(inferences)

# Create the accuracy metric
accuracy_perclass = Accuracy(num_classes=4, task='multiclass', average='none')
accuracy_micro = Accuracy(num_classes=4, task='multiclass', average='micro')
accuracy_macro = Accuracy(num_classes=4, task='multiclass', average='macro')
accuracy_ignorebg = Accuracy(num_classes=4, task='multiclass', average='micro', ignore_index=0)

# Compute the accuracy between labels and inferences
accuracy_perclass_value = accuracy_perclass(inferences_tensor, labels_tensor)
accuracy_micro_value = accuracy_micro(inferences_tensor, labels_tensor)
accuracy_macro_value = accuracy_macro(inferences_tensor, labels_tensor)
accuracy_ignorebg_value = accuracy_ignorebg(inferences_tensor, labels_tensor)

# Create the IoU metric
iou_perclass = JaccardIndex(num_classes=4, task='multiclass', average='none')
iou_micro = JaccardIndex(num_classes=4, task='multiclass', average='micro')
iou_macro = JaccardIndex(num_classes=4, task='multiclass', average='macro') 
iou_ignorebg = JaccardIndex(num_classes=4, task='multiclass', average='micro', ignore_index=0)

# Compute the mIoU between labels and inferences
iou_perclass_value = iou_perclass(inferences_tensor, labels_tensor)
iou_micro_value = iou_micro(inferences_tensor, labels_tensor)
iou_macro_value = iou_macro(inferences_tensor, labels_tensor)
iou_ignorebg_value = iou_ignorebg(inferences_tensor, labels_tensor)

# Print the accuracy and mIoU values
print('Accuracy per class:', accuracy_perclass_value)
print('Accuracy micro:', accuracy_micro_value)
print('Accuracy macro:', accuracy_macro_value)
print('Accuracy ignore bg:', accuracy_ignorebg_value)
print('IoU per class:', iou_perclass_value)
print('IoU micro:', iou_micro_value)
print('IoU macro:', iou_macro_value)
print('IoU ignore bg:', iou_ignorebg_value)

Accuracy per class: tensor([0.6286, 0.3713, 0.5654, 0.6672])
Accuracy micro: tensor(0.6140)
Accuracy macro: tensor(0.5581)
Accuracy ignore bg: tensor(0.5391)
IoU per class: tensor([0.5798, 0.2158, 0.1068, 0.3132])
IoU micro: tensor(0.4430)
IoU macro: tensor(0.3039)
IoU ignore bg: tensor(0.5244)
